# Eigenvector Centrality

In this notebook, we will compute the Eigenvector centrality for vertice in our test database using cuGraph and NetworkX. The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware |
| --------------|------------|------------------|-----------------|----------------|
| Don Acosta    | 07/05/2022 | created          | 22.08 nightly   | DGX Tesla V100 CUDA 11.5
| Ralph Liu    | 07/11/2023 | updated          | 23.08 nightly   | DGX Tesla V100 CUDA 12.0

## Introduction
Eigenvector centrality computes the centrality for a vertex based on the
centrality of its neighbors. The Eigenvector of a node measures influence within a graph by taking into account a vertex's connections to other highly connected vertices.


See [Eigenvector Centrality on Wikipedia](https://en.wikipedia.org/wiki/Eigenvector_centrality) for more details on the algorithm.



For a graph with V vertices where $A = (A_v,t)$ is the adjacency matrix, the Eigenvector centrality score of vertex $v$ is defined as


$x_v = \frac{1}{\lambda}{\sum_{t \in M(v)}}x_t = \frac{1}{\lambda}{\sum_{t \in V}{a_v,x_t}}$

Where $M(v)$ is the neighbor set for $v$ and λ is a constant.

[Learn more about EigenVector Centrality](https://www.sci.unich.it/~francesc/teaching/network/eigenvector.html)


To compute the Eigenvector centrality scores for a graph in cuGraph we use:<br>
__df = cugraph.eigenvector_centrality(G, max_iter=100, tol=1.0e-6, normalized=True)__

    G : cuGraph.Graph or networkx.Graph
    max_iter : int, optional (default=100)
        The maximum number of iterations before an answer is returned. This can
        be used to limit the execution time and do an early exit before the
        solver reaches the convergence tolerance.
    tol : float, optional (default=1e-6)
        Set the tolerance the approximation, this parameter should be a small
        magnitude value.
        The lower the tolerance the better the approximation. If this value is
        0.0f, cuGraph will use the default value which is 1.0e-6.
        Setting too small a tolerance can lead to non-convergence due to
        numerical roundoff. Usually values between 1e-2 and 1e-6 are
        acceptable.
    normalized : bool, optional, default=True
        If True normalize the resulting eigenvector centrality values

   

Returns:

    df : cudf.DataFrame or Dictionary if using NetworkX
        GPU data frame containing two cudf.Series of size V: the vertex
        identifiers and the corresponding eigenvector centrality values.
        df['vertex'] : cudf.Series
            Contains the vertex identifiers
        df['eigenvector_centrality'] : cudf.Series
            Contains the eigenvector centrality of vertices


#### Some notes about vertex IDs...
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../../img/zachary_black_lines.png)


Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf

In [ ]:
# NetworkX libraries
import networkx as nx
import pandas as pd

### Some Prep

In [ ]:
from cugraph.datasets import karate

Method to show most influencial nodes

In [ ]:
def topKVertices(eigen, col, k):
    top = eigen.nlargest(n=k, columns=col)
    top = top.sort_values(by=col, ascending=False)
    return top


### Create a Graph

cuGraph's dataset objects depend on cuDF for data loading and the intial edge-list creation.
The original data file contains an edge-list, which represents the connection of a vertex to another. These `source` to `destination` pairs are in what is known as Coordinate Format (COO).

In [ ]:
karate.get_edgelist(download=True)

In [ ]:
G = karate.get_graph()

### Call the Eigenvector Centrality algorithm

In [ ]:
# Call cugraph.eigenvector_centrality 
k_df = cugraph.eigenvector_centrality(G, max_iter=1000)

_It was that easy!_  


---

### Now compute using NetworkX

In [ ]:
# Define the path to the test data  
datafile='../../data/karate-data.csv'

# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

### Compute and Display the top 5 eigenvector centrality values with their nodes in NetworkX.

In [ ]:
nx_evc = nx.eigenvector_centrality(Gnx)
evc_nx_sv = sorted(((value, key) for (key,value) in nx_evc.items()), reverse=True)
evc_nx_sv[:5]

### Display the top 5 from the earlier cugraph-based computation.

In [ ]:
k_df = k_df.rename(columns={"eigenvector_centrality": "cu_eigen"}, copy=False)
k_df = k_df.sort_values("cu_eigen").reset_index(drop=True)
topKVertices(k_df, "cu_eigen", 5)

As mentioned, the scores are slightly different but the ranking is the same.

___
Copyright (c) 2022-2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___